<a href="https://colab.research.google.com/github/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/defs_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2 as cv
import numpy as np

In [0]:
def convert_to_grayscale(image):
  return cv.cvtColor(image, cv.COLOR_RGB2GRAY)

In [0]:
def threshold_image(image, method, block_size, bias):
  # convert relative block size to absolute block size
  width, height = image.shape
  block_size = int(block_size * min(width, height))
  # if block_size is even, we must make it uneven (see def. of adaptiveThreshold)
  if block_size % 2 == 0:
    block_size += 1
  return cv.adaptiveThreshold(image, 255, method, cv.THRESH_BINARY, block_size, bias)

In [0]:
def find_contours(image):
  # image must be inverted, because findContours treats white as foreground and black as background
  # if we don't invert the image, the image border will be detected as a contour
  image = cv.bitwise_not(image)
  contours, _ = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
  return contours

In [0]:
def draw_contours(image, contours, thickness=0.01, color=(0, 0, 255)):
  # convert image from grayscale to RGB
  # necessary because we draw RGB contours
  image = cv.cvtColor(image, cv.COLOR_GRAY2RGB)
  height, width = image.shape[:2]
  thickness = (height + width) / 2 * thickness
  # draw contours using OpenCV
  if not any(x is None for x in contours):
    cv.drawContours(image, contours, -1, color, int(thickness))
  return image

In [0]:
def approximate_contour(contour, precision):
  # convert relative precision to absolute precision (epsilon)
  _, _, width, height = cv.boundingRect(contour)
  epsilon = (width + height) / 2 * precision
  return cv.approxPolyDP(contour, epsilon, closed=True)

In [0]:
def get_foursided_contours(contours):
  return [contour for contour in contours if len(contour) == 4]

In [0]:
def get_largest_contour(contours):
  lengths = [cv.contourArea(x) for x in contours]
  longest = np.argmax(lengths)
  return contours[longest]

In [0]:
def check_sudoku_contour(image, contour, min_area):
  if contour is None:
    return False
  width, height = image.shape[:2]
  total_image_area = width * height
  contour_area = cv.contourArea(contour)
  return contour_area / total_image_area > min_area